In [1]:
import re
import nltk
import numpy as np

In [2]:
from keras.preprocessing.text import one_hot
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout, Activation, Embedding

Using TensorFlow backend.


In [3]:
np.random.seed(7)

## Tag the training data

In [4]:
def clean_sentence(s):
    c = s.lower().strip()
    return re.sub('[^a-z ]', '', c)

In [5]:
def translate_word_class(tag):
    if tag in ['JJ', 'JJR', 'JJS']:
        return 1 #'adjective'
    if tag in ['NN', 'NNS', 'NNP', 'NNPS']:
        return 2 #'noun'
    if tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
        return 3 #'verb'
    if tag in ['CC', 'IN']:
        return 4 #'link'
    return 5 #'other'

In [6]:
def map_tagging(sentence):
    translation = list()
    for word, tag in sentence:
        translation.append((word, translate_word_class(tag)))
    return translation

In [7]:
def map_encoding(sentence):
    encoded = list()
    for word, tag in sentence:
        encodedInt = one_hot(word,300000)[0]
        encoded.append([encodedInt,tag])
    return encoded        

In [8]:
END = 6
def sentence_labeling(sentence):
    labels = list()
    for word, tag in sentence[1:]:
        labels.append(tag)
    labels.append(END)
    return labels

In [9]:
def tag_words(sentences):
    tagged_words = list()
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        tagged_words.append(nltk.pos_tag(words))
    return tagged_words

In [10]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [11]:
train_sentences = list()
with open('./data_set/training_set70.txt') as train:
    for line in train:
        train_sentences.append(clean_sentence(line))

test_sentences = list()
with open('./data_set/test_set25.txt') as train:
    for line in train:
        test_sentences.append(clean_sentence(line))

In [12]:
train_sentences[:5]

['correct', 'dick agreed', 'sorry wrong', 'where is mary', 'he is happy now']

In [13]:
tagged_sentences = tag_words(train_sentences)
tagged_sentences_test = tag_words(test_sentences)

In [14]:
tagged_sentences[:5]
tagged_sentences_test[:5]

[[('sam', 'JJ'), ('didnt', 'NNS'), ('like', 'IN'), ('elections', 'NNS')],
 [('dick', 'NN'),
  ('and', 'CC'),
  ('sam', 'NN'),
  ('were', 'VBD'),
  ('disappointed', 'JJ')],
 [('where', 'WRB'), ('is', 'VBZ'), ('my', 'PRP$'), ('apple', 'NN')],
 [('yes', 'RB'), ('thats', 'NNS'), ('correct', 'VBP')],
 [('no', 'DT'), ('its', 'PRP$'), ('wrong', 'NN')]]

In [15]:
my_tagged_sentences = list(map(map_tagging, tagged_sentences))
test_tagged_sentences = list(map(map_tagging, tagged_sentences_test))

In [16]:
my_tagged_sentences[:5]

[[('correct', 2)],
 [('dick', 2), ('agreed', 3)],
 [('sorry', 2), ('wrong', 1)],
 [('where', 5), ('is', 3), ('mary', 1)],
 [('he', 5), ('is', 3), ('happy', 1), ('now', 5)]]

In [17]:
x = flatten(map(map_encoding, my_tagged_sentences))
test = flatten(map(map_encoding, test_tagged_sentences))

In [19]:
def create_matrix(numbers,size,var_index):
    matrix = np.zeros((size,1))
    sub_matrix = np.zeros((1))
    i = 0
    for number in numbers:
        sub_matrix[0] = number[var_index]
        matrix[i][0] = sub_matrix
        i=i+1
    return matrix

In [20]:
x_train = create_matrix(x,len(x),0)
y_train = create_matrix(x,len(x),1)
x_test = create_matrix(test,len(test),0)
y_test = create_matrix(test,len(test),1)
print(x_train[:3])
print(y_train[:3])
print(len(x_train))
print(len(y_train))
print(x_test[:3])
print(y_test[:3])
print(len(x_test))
print(len(y_test))

[[  15753.]
 [  51163.]
 [ 121689.]]
[[ 2.]
 [ 2.]
 [ 3.]]
328
328
[[ 228419.]
 [ 233806.]
 [ 214517.]]
[[ 1.]
 [ 2.]
 [ 4.]]
132
132


In [21]:
print('Build model...')
model = Sequential()
model.add(Embedding(300000, 128, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train, batch_size=32, nb_epoch=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            32)
print('Test score:', score)
print('Test accuracy:', acc)

Build model...
Train...


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/tensorflow/python/ops/gradients.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 328 samples, validate on 132 samples
Epoch 1/15
328/328 [==============================] - 31s - loss: 0.6219 - acc: 0.0671 - val_loss: 0.5071 - val_acc: 0.0606
Epoch 2/15
328/328 [==============================] - 29s - loss: 0.3935 - acc: 0.0793 - val_loss: 0.2220 - val_acc: 0.0606
Epoch 3/15
328/328 [==============================] - 29s - loss: 0.0175 - acc: 0.0793 - val_loss: -0.2676 - val_acc: 0.0606
Epoch 4/15
328/328 [==============================] - 30s - loss: -0.6218 - acc: 0.0793 - val_loss: -1.1053 - val_acc: 0.0606
Epoch 5/15
328/328 [==============================] - 29s - loss: -1.8197 - acc: 0.0793 - val_loss: -2.5462 - val_acc: 0.0606
Epoch 6/15
328/328 [==============================] - 28s - loss: -3.6793 - acc: 0.0793 - val_loss: -4.8093 - val_acc: 0.0606
Epoch 7/15
328/328 [==============================] - 29s - loss: -6.7291 - acc: 0.0793 - val_loss: -8.3634 - val_acc: 0.0606
Epoch 8/15
328/328 [==============================] - 29s - loss: -10.8728 - 

In [1]:
"y = flatten(map(sentence_labeling, my_tagged_sentences))"

NameError: name 'flatten' is not defined

In [122]:
"y[:5]"

[6, 3, 6, 1, 6]